#### Summary:
- 3 baselines, copy the value from last period, copy the value from the same period the year before, take the max between the last 2 periods
- Models on geo features alone can at best match the performance of the baseline
- Need to find a way to bring some improvement by adding conflict features!!

# Gdelt events data processing

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import os
import requests
import zipfile
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup
import helper_functions.download_gdelt_events as download_gdelt
from helper_functions.download_gdelt_events import consolidate_files
from helper_functions.gdelt_data_mapping_optimized import load_gadm_data, process_gdelt_data, consolidate_and_merge_fews
import multiprocessing
import nest_asyncio
nest_asyncio.apply()
from helper_functions.url_scraping_base import process_urls_in_chunks
import numpy as np
import glob
import re
import collections.abc

# ---------------------------------------------------------------------
# Download GDELT raw data
# ---------------------------------------------------------------------
start_date = datetime(2016, 1, 1)
end_date = datetime(2024, 2, 28)
save_directory = "../data/gdelt/events/1_raw"
output_file = "../data/gdelt/events/2_consolidated/combined_data.parquet"

# Paths
gadm_path = "../data/gadm/gadm_410_filtered_v2.gpkg"
fews_path = "../data/fews/fews_with_conflicts_admin2.parquet"
gdelt_path = "../data/gdelt/events/2_consolidated/combined_data.parquet"
output_dir = "../data/gdelt/events/3_mapped/"
final_output_path = os.path.join(output_dir, "gdelt_mapped.parquet")

# print("Downloading GDELT data...")
# download_gdelt.download_gdelt_data(start_date, end_date, save_directory, max_workers=8)

# print("Consolidating GDELT data...")
# consolidate_files(save_directory, output_file)
# 4M articles at this point

# recommended_cpus = max(1, multiprocessing.cpu_count() - 1)

# # # ---------------------------------------------------------------------
# # Load reference data
# # ---------------------------------------------------------------------
# fews_df = pd.read_parquet(fews_path)
# fews_df = fews_df[['ADMIN0', 'ADMIN1', 'ADMIN2', 'period', 'CS_score']]


# print("Loading GADM")
# gadm_gdf, fews_df = load_gadm_data(gadm_path, fews_df)

# # ---------------------------------------------------------------------
# # Load and process GDELT
# # ---------------------------------------------------------------------
# gdelt_df = pd.read_parquet(gdelt_path)
# gdelt_df = gdelt_df.dropna(subset=["ActionGeo_Lat", "ActionGeo_Long"])

# print("Processing GDELT data")
#
#  process_gdelt_data(gdelt_df, gadm_gdf, output_dir, num_cpus=recommended_cpus)

# # ---------------------------------------------------------------------
# # Merge with FEWS
# # ---------------------------------------------------------------------
# print("Consolidating mapped GDELT data with FEWS data")

# df_final = consolidate_and_merge_fews(mapped_gdelt_dir=output_dir, fews_df=fews_df)
# # 4M articles at this point

# df_final.to_parquet(final_output_path, index=False)

# print("Number of records in final dataset:", len(df_final))
# print("\nFirst few rows of the final dataset:")
# df_final.head()
# 4M articles at this point


In [ ]:
df = pd.read_parquet(final_output_path)
# df = df[df['period'] == '202402']
# ---------------------------------------------------------------------
# Extract unique URLs
# ---------------------------------------------------------------------

def flatten_and_filter(source_col):
    urls = set()
    for item in source_col.dropna():
        if isinstance(item, (list, tuple, set, np.ndarray)):  # 👈 added np.ndarray
            urls.update(
                x for x in item
                if isinstance(x, str) and x.startswith("http")
            )
    return urls

unique_urls = sorted(flatten_and_filter(df["SOURCEURL"]))

# ---------------------------------------------------------------------
# Handle already processed chunks
# ---------------------------------------------------------------------
chunk_dir = "../data/gdelt/events/scraped_urls"
os.makedirs(chunk_dir, exist_ok=True)
parquet_files = glob.glob(os.path.join(chunk_dir, "chunk_*.parquet"))

chunk_numbers = (
    [int(re.search(r'chunk_(\d+)\.parquet', f).group(1)) for f in parquet_files]
    if parquet_files else []
)
highest_chunk = max(chunk_numbers) + 1 if chunk_numbers else 0
print(f"Highest chunk number: {highest_chunk}")

if parquet_files:
    processed = pd.concat([pd.read_parquet(f) for f in parquet_files], ignore_index=True)
    processed_urls = set(processed['url'].dropna().values)
else:
    processed_urls = set()

# Remove already processed URLs
unique_urls = [u for u in unique_urls if u not in processed_urls]

print(f"Remaining URLs to process: {len(unique_urls)}")

# ---------------------------------------------------------------------
# Run URL scraping
# ---------------------------------------------------------------------
process_urls_in_chunks(
    urls=unique_urls,
    chunk_size=10000,
    concurrency=150,
    chunk_id=highest_chunk,
    timeout=4,
    max_retries=3,
    max_selenium_workers=4,
    fallback_mode="async_only",
    output_dir=chunk_dir
)
# 1.9M articles at this point

### Clean the scraped URLS, remove duplicate articles, and articles not related to FS

In [ ]:
import re
import os
import math
import logging
import multiprocessing as mp
from collections import Counter

from tqdm.notebook import tqdm
import pandas as pd
import glob
import spacy

from helper_functions.topic_modelling.text_processing_parallel import preprocess_text_parallel
from helper_functions.topic_modelling.flatten_articles import filter_articles_by_lexicon
from helper_functions.topic_modelling.deduplication import deduplicate_minhash
from helper_functions.run_ner_parallel import run_ner_parallel, inject_countries_from_demonyms  # ✅ imported correctly

# ----------------- Load scraped files -----------------
parquet_dir = "../data/gdelt/events/scraped_urls"
parquet_files = glob.glob(os.path.join(parquet_dir, "chunk_*.parquet"))
if not parquet_files:
    raise FileNotFoundError(f"No parquet files found in {parquet_dir}")

df_scraped = pd.concat([pd.read_parquet(f) for f in parquet_files], ignore_index=True)
print("Total scraped articles:", len(df_scraped))
# ~1.9M articles at this point

# ----------------- Remove noisy events -----------------
mapped_path = "../data/gdelt/events/2_consolidated/combined_data.parquet"
if not os.path.exists(mapped_path):
    raise FileNotFoundError(f"Missing mapped GDELT file: {mapped_path}")

codes_mapping = pd.read_parquet(mapped_path)
codes_mapping = (
    codes_mapping
    .groupby("SOURCEURL", as_index=False)
    .agg({"EventCode": "max"})
)

df_scraped = pd.merge(df_scraped, codes_mapping, left_on='url', right_on='SOURCEURL', how='inner')
df_scraped = df_scraped[~df_scraped["EventCode"].isin([10, 20, 30, 31, 32, 33, 34, 42, 43, 46])].reset_index(drop=True)
print("Remaining after removing 10/20/30/31/32/33/34/42/43/46:", len(df_scraped))
# ~920K here

# ----------------- Pre-processing pipeline -----------------
df_scraped['header'] = df_scraped['header'].fillna("").astype(str)
df_scraped['body'] = df_scraped['body'].fillna("").astype(str)
df_scraped['text'] = df_scraped['header'] + " " + df_scraped['body']

print("Applying LEAP4FNSSA lexicon filter (raw text)...")

df_scraped = filter_articles_by_lexicon(
    df_scraped,
    clean_text_col="text",
    lexicon_path="../data/LEAP4FNSSA_LEXICON_long.csv"
)
print("Lexicon filtering complete. Remaining:", df_scraped.shape)
# ~381k articles

# 2️⃣ Heavy preprocessing
df_scraped = preprocess_text_parallel(df_scraped, text_col='text')
print("Text preprocessing done.")

# 3️⃣ Truncate to 500 words
def truncate_to_500_words(text):
    words = str(text).split()
    return ' '.join(words[:500])

df_scraped['clean_text'] = df_scraped['clean_text'].apply(truncate_to_500_words)

# 4️⃣ Deduplicate by near-duplicate text
df_scraped = deduplicate_minhash(df_scraped, text_col='clean_text', threshold=0.85)
print("After deduplication:", df_scraped.shape)
# ~354k articles

# ----------------- NER Extraction -----------------
print("🔍 Running NER location extraction...")

# Inject country names for demonyms before NER
df_scraped["clean_text"] = df_scraped["clean_text"].apply(inject_countries_from_demonyms)

# Automatically disable multiprocessing in Jupyter (for stability)
n_process = 1 if "ipykernel" in mp.current_process().name.lower() else mp.cpu_count()

df_scraped = run_ner_parallel(df_scraped, text_col="clean_text", n_process=n_process)
print("✅ NER extraction complete.")

# ----------------- Load FEWS countries (for both refinement & filtering) -----------------
fews_path = "../data/fews/fews_with_conflicts_admin2.parquet"
fews_df = pd.read_parquet(fews_path)
fews_countries = [country.lower() for country in fews_df['ADMIN0'].unique()]

# ----------------- Refine main country using FEWS mentions -----------------
# Keep raw spaCy outputs for debugging if needed
df_scraped["NER_admin0_raw"] = df_scraped["NER_admin0"]
df_scraped["NER_admin1_raw"] = df_scraped["NER_admin1"]
df_scraped["NER_admin2_raw"] = df_scraped["NER_admin2"]

def pick_main_country_from_text(text: str) -> str:
    """
    Count mentions of each FEWS country in the article text (clean_text)
    and return the most frequently mentioned one (or None if no FEWS country appears).
    """
    if not isinstance(text, str):
        return None

    txt = text.lower()
    counts = Counter()

    # naive substring counting; demonyms already injected as country names
    for country in fews_countries:
        c = txt.count(country)
        if c > 0:
            counts[country] += c

    if counts:
        return counts.most_common(1)[0][0]
    return None

def refine_ner_country(row):
    main = pick_main_country_from_text(row["clean_text"])
    if main:
        # override top-level country with the most-mentioned FEWS country
        row["NER_admin0"] = main
    return row

df_scraped = df_scraped.apply(refine_ner_country, axis=1)
print("✅ NER country refinement based on FEWS mentions complete.")

# ----------------- Filter by FEWS countries -----------------
df_scraped = df_scraped[
    df_scraped['NER_admin0'].isin(fews_countries) |
    df_scraped['NER_admin1'].isin(fews_countries) |
    df_scraped['NER_admin2'].isin(fews_countries)
]
print("Remaining after FEWS NER filter:", len(df_scraped))

# ----------------- Filter by crisis terms -----------------
crisis_terms = re.compile(
    r"(famine|hunger|malnutrition|food|nutrition|crop|harvest|yield|farmer|"
    r"agricultur|drought|flood|rainfall|storm|cyclone|heatwave|disaster|aid|relief|"
    r"refugee|displaced|idp|conflict|war|violence|attack|protest|unrest|"
    r"livelihood|poverty|shortage|inflation|market|commodity|price|"
    r"food security|early warning|ipc|wfp|unicef|ocha|ngo)",
    re.I
)

df_scraped = df_scraped[
    df_scraped["clean_text"].str.contains(crisis_terms, na=False)
]

print("After crisis-context filter:", len(df_scraped))

# ----------------- Save output -----------------
out_path = "../data/gdelt/events/scraped_urls/cleaned_filtered_urls.parquet"
df_scraped.to_parquet(out_path, index=False)
print(f"💾 Saved to {out_path}")

In [ ]:
df_scraped[df_scraped['url'] == 'https://www.thenews.com.pk/print/666155-30-killed-in-eastern-burkina-faso-attack']